<a href="https://colab.research.google.com/github/arielwendichansky/LinkedIn_analysis/blob/main/linkedin_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# new


In [ ]:
!pip install linkedin-jobs-scraper


In [ ]:
import logging
from linkedin_jobs_scraper import LinkedinScraper
from linkedin_jobs_scraper.events import Events, EventData, EventMetrics
from linkedin_jobs_scraper.query import Query, QueryOptions, QueryFilters
from linkedin_jobs_scraper.filters import RelevanceFilters, TimeFilters, TypeFilters, ExperienceLevelFilters, OnSiteOrRemoteFilters

In [ ]:
job_data = []
logging.basicConfig(level=logging.INFO)

def on_data(data: EventData):
    print('[ON_DATA]', data.title, data.company, data.company_link, data.date, data.link, data.insights, len(data.description))
    job_data.append([data.title, data.company, data.company_link, data.date, data.link, data.insights, data.description])

def on_error(error):
    print('[ON_ERROR]', error)

def on_end():
    print('[ON_END]')


In [ ]:
scraper = LinkedinScraper(
    chrome_executable_path=None,
    chrome_options=None,
    headless=True,
    max_workers=1,
    slow_mo=0.5,
    page_load_timeout=40
)

INFO:li:scraper:('Using strategy AnonymousStrategy',)


In [ ]:
queries = [
    Query(
        options=QueryOptions(
            limit=1
        )
    ),
    Query(
        query='Data analyst',
        options=QueryOptions(
            locations=["Tel Aviv-Yafo, Tel Aviv District, Israel"],
            apply_link=True,
            skip_promoted_jobs=True,
            page_offset=9,
            limit=200,
            filters=QueryFilters(
                relevance=RelevanceFilters.RECENT,
                time=TimeFilters.MONTH,
                type=[TypeFilters.FULL_TIME, TypeFilters.INTERNSHIP],
                on_site_or_remote=[OnSiteOrRemoteFilters.REMOTE],
                experience=[ExperienceLevelFilters.ENTRY_LEVEL, ExperienceLevelFilters.INTERNSHIP]
            )
        )
    ),
]

In [ ]:
scraper.on(Events.DATA, on_data)
scraper.on(Events.ERROR, on_error)
scraper.on(Events.END, on_end)

scraper.run(queries)

INFO:li:scraper:('Starting new query', "Query(query= options=QueryOptions(limit=1 locations=['Worldwide'] apply_link=False skip_promoted_jobs=False page_offset=0))")
INFO:li:scraper:('Chrome debugger url', 'http://localhost:33527')
INFO:li:scraper:('Websocket debugger url: ', 'ws://localhost:33527/devtools/page/4E0B995323980F27B3F2969236984FC2')
INFO:li:scraper:('[][Worldwide]', 'Opening https://www.linkedin.com/jobs/search?location=Worldwide')
INFO:li:scraper:('[][Worldwide]', 'Trying first selectors set')
INFO:li:scraper:('[][Worldwide]', 'Trying second selectors set')
INFO:li:scraper:('[][Worldwide]', 'OK')
INFO:li:scraper:('[][Worldwide]', 'Starting pagination loop')
INFO:li:scraper:('[][Worldwide]', 'Found 60 jobs')
INFO:li:scraper:('[][Worldwide][1]', 'Processed')


[ON_DATA] Staf Mutu Dan Akreditasi Hermina Hospital Group  2024-08-11 https://id.linkedin.com/jobs/view/staf-mutu-dan-akreditasi-at-hermina-hospital-group-3999165826?position=1&pageNum=0&refId=eJ0RJE8XCtEHPR2Wp2tLEw%3D%3D&trackingId=QYsfTptRHhgLC%2B0Amv6xrg%3D%3D&trk=public_jobs_jserp-result_search-card [] 508
[ON_END]


INFO:li:scraper:('Starting new query', "Query(query=Data analyst options=QueryOptions(limit=200 locations=['Tel Aviv-Yafo'] filters=QueryFilters(relevance=RelevanceFilters.RECENT time=TimeFilters.MONTH type=[<TypeFilters.FULL_TIME: 'F'>, <TypeFilters.INTERNSHIP: 'I'>] experience=[<ExperienceLevelFilters.ENTRY_LEVEL: '2'>, <ExperienceLevelFilters.INTERNSHIP: '1'>] on_site_or_remote=[<OnSiteOrRemoteFilters.REMOTE: '2'>]) apply_link=True skip_promoted_jobs=True page_offset=9))")
INFO:li:scraper:('Chrome debugger url', 'http://localhost:40739')
INFO:li:scraper:('Websocket debugger url: ', 'ws://localhost:40739/devtools/page/B77242D5CF35FB6BD3E6713D735DE585')
INFO:li:scraper:('[Data analyst][Tel Aviv-Yafo]', 'Opening https://www.linkedin.com/jobs/search?keywords=Data+analyst&location=Tel+Aviv-Yafo&sortBy=DD&f_TPR=r2592000&f_JT=F%2CI&f_E=2%2C1&start=0')
INFO:li:scraper:('[Data analyst][Tel Aviv-Yafo]', 'Trying first selectors set')
INFO:li:scraper:('[Data analyst][Tel Aviv-Yafo]', 'Trying se

[ON_END]


In [ ]:
# Create a df out of the job_data list
df = pd.DataFrame(job_data, columns=['title', 'company', 'company_link', 'date', 'link', 'insights', 'description'])
df

,title,company,company_link,date,link,insights,description
0,Staf Mutu Dan Akreditasi,Hermina Hospital Group,,2024-08-11,https://id.linkedin.com/jobs/view/staf-mutu-da...,[],Staf Mutu dan Akreditasi\n\nDibutuhkan Staf Mu...


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         0 non-null      object
 1   company       0 non-null      object
 2   company_link  0 non-null      object
 3   date          0 non-null      object
 4   link          0 non-null      object
 5   insights      0 non-null      object
 6   description   0 non-null      object
dtypes: object(7)
memory usage: 124.0+ bytes


# Number applicants

In [ ]:
#Import dependencies
import requests
from bs4 import BeautifulSoup
import random
import pandas as pd

In [ ]:
title = "Data Analyst"  # Job title
location = "Tel Aviv-Yafo, Tel Aviv District, Israel"  # Job location
start = 0  # Starting point for pagination

In [ ]:
# Construct the URL for LinkedIn job search
list_url = f"https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords={title}&location={location}&start={start}"

# Send a GET request to the URL and store the response
response = requests.get(list_url)

#Get the HTML, parse the response and find all list items(jobs postings)
list_data = response.text
list_soup = BeautifulSoup(list_data, "html.parser")
page_jobs = list_soup.find_all("li")

In [ ]:
#Create an empty list to store the job postings
id_list = []

In [ ]:
#Itetrate through job postings to find job ids
for job in page_jobs:
    base_card_div = job.find("div", {"class": "base-card"})
    job_id = base_card_div.get("data-entity-urn").split(":")[3]
    print(job_id)
    id_list.append(job_id)

print(id_list)

3988136365
3987818560
3964922449
3982506529
3945779355
3903122148
3958870589
3967190464
3938469817
3960872947
['3988136365', '3987818560', '3964922449', '3982506529', '3945779355', '3903122148', '3958870589', '3967190464', '3938469817', '3960872947']


In [ ]:
# Initialize an empty list to store job information
job_list = []

# Loop through the list of job IDs and get each URL
for job_id in id_list:
    # Construct the URL for each job using the job ID
    job_url = f"https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{job_id}"

    # Send a GET request to the job URL and parse the reponse
    job_response = requests.get(job_url)
    print(job_response.status_code)
    job_soup = BeautifulSoup(job_response.text, "html.parser")

     # Create a dictionary to store job details
    job_post = {}

    # Try to extract and store the job title
    try:
        job_post["job_title"] = job_soup.find("h2", {"class":"top-card-layout__title font-sans text-lg papabear:text-xl font-bold leading-open text-color-text mb-0 topcard__title"}).text.strip()
    except:
        job_post["job_title"] = None

    # Try to extract and store the company name
    try:
        job_post["company_name"] = job_soup.find("a", {"class": "topcard__org-name-link topcard__flavor--black-link"}).text.strip()
    except:
        job_post["company_name"] = None

    # Try to extract and store the time posted
    try:
        job_post["time_posted"] = job_soup.find("span", {"class": "posted-time-ago__text topcard__flavor--metadata"}).text.strip()
    except:
        job_post["time_posted"] = None

    # Try to extract and store the number of applicants
    try:
        job_post["num_applicants"] = job_soup.find("span", {"class": "num-applicants__caption topcard__flavor--metadata topcard__flavor--bullet"}).text.strip()
    except:
        job_post["num_applicants"] = None


    # Append the job details to the job_list
    job_list.append(job_post)

200
429
200
429
200
429
200
200
429
429


In [ ]:
#Check if the list contains all the desired data
job_list

[{'job_title': 'Junior Data Analyst',
  'company_name': 'OpenWeb',
  'time_posted': '1 week ago',
  'num_applicants': None},
 {'job_title': None,
  'company_name': None,
  'time_posted': None,
  'num_applicants': None},
 {'job_title': 'Data Analyst',
  'company_name': 'Sorbet',
  'time_posted': '1 month ago',
  'num_applicants': None},
 {'job_title': None,
  'company_name': None,
  'time_posted': None,
  'num_applicants': None},
 {'job_title': 'Data Analyst',
  'company_name': 'Flare',
  'time_posted': '6 days ago',
  'num_applicants': None},
 {'job_title': None,
  'company_name': None,
  'time_posted': None,
  'num_applicants': None},
 {'job_title': 'Data Analyst',
  'company_name': 'Octopol',
  'time_posted': '1 month ago',
  'num_applicants': None},
 {'job_title': 'Data Analyst',
  'company_name': 'Tailor Brands',
  'time_posted': '2 months ago',
  'num_applicants': None},
 {'job_title': None,
  'company_name': None,
  'time_posted': None,
  'num_applicants': None},
 {'job_title': N

In [ ]:
# Create a pandas DataFrame using the list of job dictionaries 'job_list'
jobs_df = pd.DataFrame(job_list)
jobs_df

,job_title,company_name,time_posted,num_applicants
0,Junior Data Analyst,OpenWeb,1 week ago,None
1,None,None,None,None
2,Data Analyst,Sorbet,1 month ago,None
3,None,None,None,None
4,Data Analyst,Flare,6 days ago,None
5,None,None,None,None
6,Data Analyst,Octopol,1 month ago,None
7,Data Analyst,Tailor Brands,2 months ago,None
8,None,None,None,None
9,None,None,None,None
